<a href="https://colab.research.google.com/github/mkbahk/AmazonBraket/blob/main/QuantumApplicationAlgorithm_QiskitNature0_7_2_GettingStarted_mkbahk_20251208.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Python 3.9 ~ 3.11 환경에서 아래 순서대로 설치
!pip install qiskit[visualization]==1.2.4
!pip install qiskit-aer==0.15.1
!pip install qiskit-algorithms==0.3.1
!pip install qiskit-nature[pyscf]==0.7.2
!pip install pyscf==2.6.0
!pip install py3Dmol

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 16.5 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=6bb6c63126136524f5a1b15eeda3c7dd8a4a83a805002841bf24ebce7b9139de
  Stored in directory: /root/.cache/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

In [4]:
# Use PySCF, a classical computational chemistry software
# package, to compute the one-body and two-body integrals in
# electronic-orbital basis, necessary to form the Fermionic operator
driver = PySCFDriver(
    #atom='H .0 .0 .0; H .0 .0 0.735',
    atom="O 0.0 0.0 0.0; H 0.757 0.586 0.0; H -0.757 0.586 0.0",
    unit=DistanceUnit.ANGSTROM,
    basis='sto3g',
)

#ElectronicStructureProblem
problem = driver.run()

print(f"Number of Particles(전자수|입자수): {problem.num_particles}")
print(f"Number of Spatial Orbitals(공간 괘도 수): {problem.num_spatial_orbitals}")
#각 공간 오비탈에 업/다운 스핀이 있으므로 보통 공간 오비탈 수의 2배 → 스핀 오비탈 수
#(스핀 궤도 수라고도 불림)
print(f"Number of Spin Orbitals(스핀 오비탈 수): {problem.num_spin_orbitals}")


Number of Particles(전자수|입자수): (5, 5)
Number of Spatial Orbitals(공간 괘도 수): 7
Number of Spin Orbitals(스핀 오비탈 수): 14


In [5]:
# setup the qubit mapper
from qiskit_nature.second_q.mappers import ParityMapper

mapper = ParityMapper(num_particles=problem.num_particles)

In [6]:
# setup the classical optimizer for the VQE
from qiskit_algorithms.optimizers import L_BFGS_B

optimizer = L_BFGS_B()

In [7]:
# setup the estimator primitive for the VQE
from qiskit.primitives import Estimator

estimator = Estimator()

In [8]:
# setup the ansatz for VQE
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

ansatz = UCCSD(
    problem.num_spatial_orbitals,
    problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
    ),
)

In [9]:
#ansatz.decompose().draw('mpl')
ansatz.decompose().draw()

┌───┐┌────────────────────────────────────────────────┐»
 q_0: ┤ X ├┤0                                               ├»
      └───┘│                                                │»
 q_1: ─────┤1                                               ├»
      ┌───┐│                                                │»
 q_2: ┤ X ├┤2                                               ├»
      └───┘│                                                │»
 q_3: ─────┤3                                               ├»
      ┌───┐│                                                │»
 q_4: ┤ X ├┤4                                               ├»
      ├───┤│                                                │»
 q_5: ┤ X ├┤5                                               ├»
      └───┘│   exp(-it (IIIIIIIYXXXX + IIIIIIZXXXXY))(t[0]) │»
 q_6: ─────┤6                                               ├»
      ┌───┐│                                                │»
 q_7: ┤ X ├┤7                                               ├»
      └───┘│                                                │»
 q_8: ─────┤8                                               ├»
      ┌───┐│                                                │»
 q_9: ┤ X ├┤9                                               ├»
      └───┘│                                                │»
q_10: ─────┤10                                              ├»
           │                                                │»
q_11: ─────┤11                                              ├»
           └────────────────────────────────────────────────┘»
«      ┌────────────────────────────────────────────────┐»
« q_0: ┤0                                               ├»
«      │                                                │»
« q_1: ┤1                                               ├»
«      │                                                │»
« q_2: ┤2                                               ├»
«      │                                                │»
« q_3: ┤3                                               ├»
«      │                                                │»
« q_4: ┤4                                               ├»
«      │                                                │»
« q_5: ┤5                                               ├»
«      │   exp(-it (IIIIIIYXXXXX + IIIIIIXXXXXY))(t[1]) │»
« q_6: ┤6                                               ├»
«      │                                                │»
« q_7: ┤7                                               ├»
«      │                                                │»
« q_8: ┤8                                               ├»
«      │                                                │»
« q_9: ┤9                                               ├»
«      │                                                │»
«q_10: ┤10                                              ├»
«      │                                                │»
«q_11: ┤11                                              ├»
«      └────────────────────────────────────────────────┘»
«      ┌────────────────────────────────────────────────┐»
« q_0: ┤0                                               ├»
«      │                                                │»
« q_1: ┤1                                               ├»
«      │                                                │»
« q_2: ┤2                                               ├»
«      │                                                │»
« q_3: ┤3                                               ├»
«      │                                                │»
« q_4: ┤4                                               ├»
«      │                                                │»
« q_5: ┤5                                               ├»
«      │   exp(-it (IIIIIIIYXXXZ + IIIIIIZXXXYI))(t[2]) │»
« q_6: ┤6                                               ├»
«      │                                                │»
« q_7: ┤7                                               ├»
«      │    

In [11]:
# set up our actual VQE instance
from qiskit_algorithms import VQE

vqe = VQE(estimator, ansatz, optimizer)
# ensure that the optimizer starts in the all-zero state which corresponds to
# the Hartree-Fock starting point
vqe.initial_point = [0] * ansatz.num_parameters

In [12]:
# prepare the ground-state solver and run it
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

algorithm = GroundStateEigensolver(mapper, vqe)

electronic_structure_result = algorithm.solve(problem)
electronic_structure_result.formatting_precision = 6

KeyboardInterrupt: 

In [ ]:
print(electronic_structure_result)